This function computes the interpolant for 

\begin{equation}
\phi' = -\dfrac{1}{2s_k}\mathbf{e}_k^T(\nabla_x S \mathbf{e}_i) \mathbf{e}_i
\end{equation}

where $ \mathbf{e}_i $ and $ \mathbf{e}_k $ respectively indicate the two (orthogonal) eigenvectors associated to $ S $ and $ s_k $ the eigenvalue corresponding to the eigenvector $ k $.
As a consequence, if we rewrite $ \mathbf{e}_i $ in polar coordinates with angle $ \phi \in \mathbb{S}^1 $ we get:

\begin{align*}
\mathbf{e}_i = \begin{pmatrix} \cos(\phi) \\ \sin(\phi) \end{pmatrix}, \quad \mathbf{e}_k = \begin{pmatrix} -\sin(\phi) \\ \cos(\phi) \end{pmatrix}
\end{align*}

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid|
| Y | array (Ny, Nx) | Y-meshgrid|
| s | float | eigenvalue $ s_k $ of rate of strain $ S $|
| Sij_x| float | spatial derivative of $ S = \begin{pmatrix} S11 && S12 \\ S12 && S22 \end{pmatrix} $ in x-direction|
| Sij_y| float | spatial derivative of $ S = \begin{pmatrix} S11 && S12 \\ S12 && S22 \end{pmatrix} $ in y-direction|  


*__phi_prime_OECS* computes $ \dot{\phi} $ over a three-dimensional meshgrid given by the two spatial components $ X, Y $ and the angle $ \phi $. 

*_phi_prime_OECS* computes the interpolant for $ \dot{\phi} $.

In [ ]:
# Import numpy
import numpy as np

# Import numba (for fast computing)
from numba import njit, prange

# Import math tools
from math import cos, sin

In [1]:
@njit(parallel=True)
def __phi_prime_OECS(s, S11_x, S11_y, S12_x, S12_y, S22_x, S22_y):
    
    '''
    Computes phi_prime over gridded domain based on the elements of the rate of strain tensor
    
    Parameters:
        s:               array(Ny, Nx), rate of attraction/repulsion
        Sij_x:           array(Ny, Nx), derivative with respect to x-th coordinate of the ij-element of the Cauchy Green strain tensor over the gridded domain 
        Sij_y:           array(Ny, Nx), derivative with respect to y-th coordinate of the ij-element of the Cauchy Green strain tensor over the gridded domain 
    
    Returns:
        phi_prime:       array(Ny, Nx, 360), contains phi_prime over gridded domain and angles
        phi:             array(360, ), angles
    '''
    
    # angles
    phi = np.linspace(0, 2*np.pi, 360)
    
    # initialize array
    phi_prime = np.zeros((S11_x.shape[0], S11_x.shape[1], phi.shape[0]))
    
    # iterate in y-direction
    for i in prange(S11_x.shape[0]):
        
        # iterate in x-direction
        for j in prange(S11_x.shape[1]):
            
            # eigenvalue at (i,j)
            s_ = s[i, j]
            
            # check if eigenvalue is finite (and not nan as this would mean that we are outside the domain)
            if abs(s_) > 0 and np.isfinite(s_+S11_x[i, j]+S11_y[i, j]+S12_x[i, j]+S12_y[i, j]+S22_x[i, j]+S22_y[i, j]):
                
                s_matrix = s_*np.eye(2, 2)
            
                S11_x_ = S11_x[i, j]
                S11_y_ = S11_y[i, j]
            
                S12_x_ = S12_x[i, j]
                S12_y_ = S12_y[i, j]
            
                S22_x_ = S22_x[i, j]
                S22_y_ = S22_y[i, j]
            
                # iterate over phi
                for p in prange(phi.shape[0]):
                
                    phi_ = phi[p]
                
                    e = np.array([cos(phi_), sin(phi_)])
                    v = np.array([-sin(phi_), cos(phi_)])
                
                    grad_S_e = np.array([[S11_x_*cos(phi_)+S11_y_*sin(phi_), S12_x_*cos(phi_)+S12_y_*sin(phi_)], [S12_x_*cos(phi_)+S12_y_*sin(phi_), S22_x_*cos(phi_)+S22_y_*sin(phi_)]])
        
                    phi_prime[i, j, p] = -1/(2*s_)*v.T@(grad_S_e@e)
                    
            else:
                
                phi_prime[i, j, :] = 0
      
    return phi_prime, phi

In [2]:
def _phi_prime_OECS(X, Y, s, S11_x, S11_y, S12_x, S12_y, S22_x, S22_y):
    
    '''
    Computes the linear interpolant of phi_prime based on the elements of the rate of strain tensor.
    
    Parameters:
        X:                           array(Ny, Nx), X-meshgrid.
        Y:                           array(Ny, Nx), Y-meshgrid.
        s:                           array(Ny, Nx), rate of attraction/repulsion.
        Sij_x:                       array(Ny, Nx), derivative with respect to x-th coordinate of the ij-element of the rate of strain tensor over the gridded domain. 
        Sij_y:                       array(Ny, Nx), derivative with respect to y-th coordinate of the ij-element of the rate of strain tensor over the gridded domain.
    
    Returns:
        Interpolant_phi_prime:       Interpolant-object for phi_prime over gridded domain and angles.
    '''
    
    from scipy.interpolate import RegularGridInterpolator
    
    phi_prime, phi = __phi_prime_OECS(s, S11_x, S11_y, S12_x, S12_y, S22_x, S22_y)
    
    Interpolant_phi_prime = RegularGridInterpolator((Y[:,0], X[0,:], phi), phi_prime)
    
    return Interpolant_phi_prime